In [15]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

import autograd.tensor as tensor
import autograd.nn as nn

from autograd.optim import SGD
from autograd.validation import MSELoss

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## References


### Forward mode & dual numbers

- https://en.wikipedia.org/wiki/Dual_number
- https://blog.demofox.org/2014/12/30/dual-numbers-automatic-differentiation/
- https://blog.demofox.org/2017/02/20/multivariable-dual-numbers-automatic-differentiation/
- https://marksaroufim.medium.com/automatic-differentiation-step-by-step-24240f97a6e6

### Backward mode & backpropagation

- https://www.jmlr.org/papers/volume18/17-468/17-468.pdf
- https://sidsite.com/posts/autodiff/
- https://github.com/karpathy/micrograd

### More on derivatives

- http://cs231n.stanford.edu/vecDerivs.pdf
- http://cs231n.stanford.edu/handouts/linear-backprop.pdf
- https://web.eecs.umich.edu/~justincj/teaching/eecs442/notes/linear-backprop.html

In [12]:
class SimpleRegNet(nn.Module):
    def __init__(self, in_shape):
        super().__init__()
        
        self.l1 = nn.Linear(in_shape, 32)
        self.l2 = nn.Linear(32, 1)
                    
    def forward(self, X):
        out = self.l1(X).relu()
        return self.l2(out)

In [16]:
from sklearn.datasets import load_boston

X_, y_ = load_boston()["data"], load_boston()["target"]
X, y = tensor.Tensor(X_), tensor.Tensor(y_).reshape(-1, 1)

In [17]:
net = SimpleRegNet(X.shape[1])
optimizer = SGD(net.parameters(), lr=1e-6)

for i in range(500):
    net.zero_grad()
        
    loss = MSELoss(net(X), y)
    loss.backward()
    optimizer.step()

print(np.sqrt(loss.value[0]))

8.873600637551323
